In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

In [120]:
data = load_boston()
data['data'].shape

/home/dex/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset

(506, 13)

In [121]:
print(data['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [122]:
def linreg_linear(X, y):
    theta = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.transpose(), X)), X.transpose()), y) #np.linalg.inv = ^(-1)
    
    ###
    z = X
    mat=np.matmul(z.transpose(),z); # product of z and z transpose
    matinv=np.linalg.inv(mat) #inverse of above product
    val=np.matmul(matinv,z.transpose()) # Product of inverse and z transpose
    theta=np.matmul(val,y) # Value of theta by multiplying value calculated above to y
    ###
    
    return theta

In [123]:
# Подготовить данные
X, y = data['data'], data['target']

X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])#добавляем столбец 1-ц в X. Нужно, чтобы сэмулировать b (y=kx+b)
#np.ones(X.shape[0]) - строим столбец из единиц
#[:, np.newaxis] - приводим его к двухмерной матрице / делаем из него столбец
#np.hstack - добавляем как еще один столбец к X

In [124]:
# Вычислить параметр theta
theta = linreg_linear(X, y)

In [125]:
theta.shape #размер вектора

(14,)

In [126]:
# Сделать предсказания для тренировочной выборки
y_pred = X.dot(theta)

In [127]:
def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')

In [128]:
# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

MSE = 21.89, RMSE = 4.68


In [129]:
# Разбить выборку на train/valid, вычислить theta,
# сделать предсказания и посчитать ошибки MSE и RMSE

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
theta = linreg_linear(X_train, y_train)
y_pred = X_valid.dot(theta)
y_train_pred = X_train.dot(theta)

In [130]:
print_regression_metrics(y_valid, y_pred)
print_regression_metrics(y_train, y_train_pred)

MSE = 29.01, RMSE = 5.39
MSE = 20.38, RMSE = 4.51


#### 3.5.3 Чему равно наибольшее стандартное отклонение у признаков?

In [133]:
# Разбить таблицу данных на матрицы X и y
X, y = data['data'], data['target']

# Добавить фиктивный столбец единиц (bias линейной модели)
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
m = X.shape[1]

In [154]:
# Проверить максимальные значения по каждому признаку в данных
X.max(axis=0)

array([  1.    ,  88.9762, 100.    ,  27.74  ,   1.    ,   0.871 ,
         8.78  , 100.    ,  12.1265,  24.    , 711.    ,  22.    ,
       396.9   ,  37.97  ])

In [165]:
ind_max = np.argmax(X.std(axis=0)) - 1  #возвращает индекс максимального значения вдоль указанной оси
#std(X) в случае одномерного массива возвращает стандартное отклонение элементов массива
#std(X) в случае двумерного массива - это вектор-строка, содержащая стандартное отклонение элементов каждого столбца
ind_max
np.max(X[ind_max])

TypeError: 'numpy.ndarray' object is not callable

In [148]:
data['feature_names'][np.argmax(X.std(axis=0)) + 1]

[11]

In [136]:
print(data['feature_names'][np.argmax(X.std(axis=0)) + 1])
print(np.max(X.std(axis=0)))#наибольшее стандартное отклонение у признаков

B
168.3704950393814


#### 3.5.4 Обучите регрессию без дополнительного столбца единиц. Какой получился RMSE? 

In [110]:
from sklearn.linear_model import LinearRegression

data = load_boston()
X, y = data['data'], data['target']

reg = LinearRegression()#this is must for the task 3.5.4
reg.fit(X, y)
y_pred = reg.predict(X)
print_regression_metrics(y, y_pred) #4.68 is the correct answer here

MSE = 21.89, RMSE = 4.68


/home/dex/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset

#### 3.5.5 Очистите данные от строк, где значение признака B меньше 50. Какой получился RMSE? 

In [180]:
#переделываем датасет в датафрейм
df = pd.DataFrame(data.data, columns=data.feature_names)

In [181]:
#в датафрейме находим индексы записей, у которых в столбце B значение меньше 50
i=df.index[df['B'] < 50].tolist()

In [182]:
#выбираем в Х признаки, а в Y целевые переменные
X, y = data['data'], data['target']

In [183]:
#удаляем из np.array строки согласно индексам из датафрейма
X = np.delete(X, i, axis=0)
y = np.delete(y, i, axis=0)

In [184]:
reg_355 = LinearRegression()
reg_355.fit(X, y)
y_pred = reg_355.predict(X)
print_regression_metrics(y, y_pred) #4.67 in the correct answer here

MSE = 21.79, RMSE = 4.67
